**Caso de Estudio con Intervalos**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
pd.set_option('display.float_format', '{:.4f}'.format)

In [ ]:
#Datos de encuestas
datos = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
datos.head()

In [ ]:
datos.shape

### Analisis de fumadores para hombre y mujeres

In [ ]:
# Comparacion empirica

In [ ]:
#Codificaremos dos variables de interes SMQ020: si es fumador o no, RIAGENDR: Genero
datos["RIAGENDRx"] = datos.RIAGENDR.replace({1: "Hombre", 2: "Mujer"})
datos["SMQ020x"] = datos.SMQ020.replace({1: "Si", 2: "No", 7: np.nan, 9: np.nan})  

In [ ]:
#Revisando las frecuencias con un atabla cruzada
datosX = datos[['RIAGENDRx', 'SMQ020x']]
pd.crosstab(datosX['RIAGENDRx'], datosX['SMQ020x'])

In [ ]:
pd.crosstab(datosX['RIAGENDRx'], datosX['SMQ020x'], normalize=True)

In [ ]:
#Obteniendo las proporciones por filas
datosZ = datosX.groupby(datosX.RIAGENDRx).agg({"SMQ020x": [lambda x: np.mean(x=="Si"), np.size]})
datosZ.columns = ["p_fila", "Total_n"] 
datosZ

In [ ]:
#Calculemos el error estandar
##Para muejres
p = datosZ.p_fila.Mujer 
n = datosZ.Total_n.Mujer 
se_mujer = np.sqrt(p * (1 - p) / n)
print(se_mujer)

#Para hombres
p = datosZ.p_fila.Hombre 
n = datosZ.Total_n.Hombre  
se_hombre = np.sqrt(p * (1 - p) / n)
print(se_hombre)

In [ ]:
#¿Cuáles son sus conclusiones sobre los errores y las proporciones?

In [ ]:
#Cálculo de los intervalos de confianza, al 95%
#Mujeres
p = datosZ.p_fila.Mujer 
n = datosZ.Total_n.Mujer 
lic = p - 1.96 * np.sqrt(p * (1 - p) / n)  
lsc = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print(lic, lsc)

In [ ]:
#Cálculo de los intervalos de confianza, al 95%
#Hombres
p = datosZ.p_fila.Hombre 
n = datosZ.Total_n.Hombre 
lic = p - 1.96 * np.sqrt(p * (1 - p) / n)  
lsc = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print(lic, lsc)

In [ ]:
#la libreria statsmodels tb los puede sacar directamente
cb_mujeres = sm.stats.proportion_confint(906, 906+2066)
cb_hombres = sm.stats.proportion_confint(1413, 1413+1340)  
print(cb_mujeres)
print(cb_hombres)

In [ ]:
#¿A que conclusiones llegampos con estos intervalos?

### Intervalos de confianza apara Muestras Independientes

In [ ]:
#Ahora que pasa con la diferencia mujeresVshombres?
datosZ.p_fila.Mujer - datosZ.p_fila.Hombre

In [ ]:
#calculo del error estándar de la diferencia de medias
se_dif = np.sqrt(se_mujer**2 + se_hombre**2)
se_dif

In [ ]:
#calculo de los limites de confianza de la diferencia de medias
dif = datosZ.p_fila.Mujer - datosZ.p_fila.Hombre
lic = dif - 1.96*se_dif
lsc = dif + 1.96*se_dif
print(lic, lsc)

### Intervalos de confianza para segmentos, edad y sexo

In [ ]:
#agrupando
datos["agegrp"] = pd.cut(datos.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])

#creando la tabla cruzada de proporcion de fumadores
pr = datos.groupby(["agegrp", "RIAGENDRx"]).agg({"SMQ020x": lambda x: np.mean(x=="Si")}).unstack()
pr.columns = ["Hombre", "Mujer"]

#creando la tabla cruzada de numero de fumadores
datosN = datos.groupby(["agegrp", "RIAGENDRx"]).agg({"SMQ020x": np.size}).unstack()
datosN.columns = ["Hombre", "Mujer"]

#error estandar para la proporcion
se = np.sqrt(pr * (1 - pr) / datosN)

#error estandar para la diferencia de fumadores en cada grupo de sexo/edad
se_diff = np.sqrt(se.Mujer**2 + se.Hombre**2)


In [ ]:
datosN

In [ ]:
se_diff

In [ ]:
# Errores estándar para la diferencia en las tasas de fumadores entre géneros, dentro de cada grupo de edad

pq = pr.Mujer - pr.Hombre
x = np.arange(pq.size)

pp = sns.scatterplot(x=x, y=pq.values, color='b')
sns.scatterplot(x=x, y=pq - 2*se_diff, color = 'g')
sns.scatterplot(x=x, y=pq + 2*se_diff, color = 'g')
pp.set_xticklabels(pq.index)
pp.set_xlabel("edad")
pp.set_ylabel("Mujer-Hombre diferencia de Proporciones")

## Intervalos de confianza para Medias, indice de masa corporal (IMC = BMI)

In [ ]:
datos.groupby("RIAGENDRx").agg({"BMXBMI": np.mean})

In [ ]:
datos.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean, np.std, np.size]})

In [ ]:
sem_mujer = 7.753 / np.sqrt(2976)
sem_hombre = 6.253 / np.sqrt(2759)
print(sem_mujer, sem_hombre)

In [ ]:
lic_mujer = 29.94 - 1.96 * 7.753 / np.sqrt(2976)
lsc_mujer = 29.94 + 1.96 * 7.753 / np.sqrt(2976)
print(lic_mujer, lsc_mujer)

In [ ]:
lic_hombre = 28.7781 - 1.96 * sem_hombre
lsc_hombre = 28.7781 + 1.96 * sem_hombre
print(lic_hombre, lsc_hombre)

In [ ]:
mujer_bmi = datos.loc[datos.RIAGENDRx=="Mujer", "BMXBMI"].dropna()
sm.stats.DescrStatsW(mujer_bmi).zconfint_mean()

### Intervalos de confianza para Diferencia de Medias (IMC = BMI)

In [ ]:
sem_dif = np.sqrt(sem_mujer**2 + sem_hombre**2)
sem_dif

In [ ]:
bmi_dif= 29.94 - 28.78
print(bmi_dif)
lcb = bmi_dif - 1.96*sem_dif
ucb = bmi_dif + 1.96*sem_dif
print(lcb, ucb)

In [ ]:
# Media, Desv est., numero de casos por grupo edad/sexo
datos["agegrp"] = pd.cut(datos.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
pr = datos.groupby(["agegrp", "RIAGENDRx"]).agg({"BMXBMI": [np.mean, np.std, np.size]}).unstack()

# Error estandar promedio para hombre y mujeres por grupo de edad
pr["BMXBMI", "sem", "Mujer"] = pr["BMXBMI", "std", "Mujer"] / np.sqrt(pr["BMXBMI", "size", "Mujer"]) 
pr["BMXBMI", "sem", "Hombre"] = pr["BMXBMI", "std", "Hombre"] / np.sqrt(pr["BMXBMI", "size", "Hombre"]) 


In [ ]:
#diferencia de IMC entre mujeeres y hokmbres en cada grupo de edad, su error estandar y los intervalos de confianza, al 95%
pr["BMXBMI", "media_dif", ""] = pr["BMXBMI", "mean", "Mujer"] - pr["BMXBMI", "mean", "Hombre"]
pr["BMXBMI", "sem_diff", ""] = np.sqrt(pr["BMXBMI", "sem", "Mujer"]**2 + pr["BMXBMI", "sem", "Hombre"]**2) 
pr["BMXBMI", "lic_dif", ""] = pr["BMXBMI", "media_dif", ""] - 1.96 * pr["BMXBMI", "sem_diff", ""] 
pr["BMXBMI", "lsc_dif", ""] = pr["BMXBMI", "media_dif", ""] + 1.96 * pr["BMXBMI", "sem_diff", ""] 



In [ ]:
#graficando las medias y sus intervalos
x = np.arange(pr.shape[0])
pp = sns.scatterplot(x=x, y=pr["BMXBMI", "media_dif", ""], color='black')
sns.scatterplot(x=x, y=pr["BMXBMI", "lic_dif", ""], color='blue')
sns.scatterplot(x=x, y=pr["BMXBMI", "lsc_dif", ""], color='blue')
pp.set_xticklabels(pr.index)
pp.set_xlabel("Edad")
pp.set_ylabel("Mujer Hombre Diferencia")